# Part 4: Interactive Guide for Agentic RAG with MCP (Local Setup)

**[EN]** This notebook is an interactive guide for demonstrating an Agentic RAG workflow on your local machine. It will guide you through setting up the Elastic MCP Server and using an external AI agent (like Claude Desktop) to perform advanced searches on Elasticsearch.<br>
**[KR]** 이 노트북은 당신의 로컬 컴퓨터에서 Agentic RAG 워크플로우를 시연하기 위한 대화형 가이드입니다. Elastic MCP 서버를 설정하고 Claude Desktop과 같은 외부 AI 에이전트를 사용하여 Elasticsearch에 대한 고급 검색을 수행하는 과정을 안내합니다.

## Part 1: Prerequisites

**[EN]** Before you begin, please ensure you have the following installed on your local machine:<br>
**[KR]** 시작하기 전에, 당신의 로컬 컴퓨터에 다음 프로그램들이 설치되어 있는지 확인해주세요:

1.  **Python**: This project is based on Python. Ensure you have a working Python environment and have installed the dependencies from `requirements.txt`.
2.  **Docker**: The new MCP Server is a Docker image. Please install Docker from [docker.com](https://www.docker.com/products/docker-desktop/).
3.  **Claude Desktop**: This demo uses Claude Desktop as the AI agent. Make sure it is installed on your machine.

## Part 2: Server Execution and Connection (Manual Guide)

**[EN]** The following steps require manual execution in your local terminal.<br>
**[KR]** 다음 단계들은 당신의 로컬 터미널에서 직접 실행해야 합니다.

### Step 2.1: Run the MCP Server using Docker

1. **Open a new terminal** in your project's root directory.
2. **Run the Docker command below.** This command will download the latest MCP server image and run it as a container. It directly passes your Elasticsearch credentials as environment variables to the container.

```bash
docker run -i --rm \
  -e ES_URL="<your-elasticsearch-url>" \
  -e ES_API_KEY="<your-api-key>" \
  -p 8889:8889 \
  docker.elastic.co/mcp/elasticsearch http
```
*(Note: Copy the values from your `elastic.env` file to fill in the `<...>` placeholders.)*

**Important**: Keep this terminal running. The server must be active for the AI agent to connect.

### Step 2.2: Configure Claude Desktop

1. Open the **Claude Desktop App**.
2. Navigate to `Settings > Developer > MCP Servers` and click **'Edit Config'**.
3. Add the following JSON configuration. This connects to the server running via Docker.

```json
{
    "mcpServers": {
        "elasticsearch-mcp-server": {
            "url": "http://localhost:8889/mcp/v1"
        }
    }
}
```
4. Save the file. Claude should now be connected to your MCP server.

## Part 3: Agentic RAG Demonstration (Interactive)

**[EN]** Now, we will simulate a customer's request. This cell will generate a complete, advanced Elasticsearch query based on your input. You will then copy this query and run it in Claude Desktop to demonstrate the Agentic RAG workflow.<br>
**[KR]** 이제 고객의 요청을 시뮬레이션합니다. 이 셀은 사용자의 입력에 따라 완전하고 진보된 Elasticsearch 쿼리를 생성합니다. 이 쿼리를 복사하여 Claude Desktop에서 실행함으로써 Agentic RAG 워크플로우를 시연합니다.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown
import json
from colpali_engine.models import ColQwen2_5, ColQwen2_5_Processor
import torch
import numpy as np

# --- Helper functions (already defined in other notebooks, included here for completeness) ---
device_map = "cpu"
if torch.backends.mps.is_available(): device_map = "mps"
elif torch.cuda.is_available(): device_map = "cuda:0"
MODEL_NAME = "tsystems/colqwen2.5-3b-multilingual-v1.0"
model = ColQwen2_5.from_pretrained(MODEL_NAME, torch_dtype=torch.bfloat16 if device_map != "cpu" else torch.float32, device_map=device_map).eval()
processor = ColQwen2_5_Processor.from_pretrained(MODEL_NAME)

def create_colqwen_query_vectors(query_text, model, processor):
    inputs = processor.process_queries([query_text]).to(model.device)
    with torch.no_grad(): outputs = model(**inputs)
    return outputs.cpu().to(torch.float32).numpy().tolist()[0]

def to_avg_vector(vectors):
    vectors_array = np.array(vectors)
    avg_vector = np.mean(vectors_array, axis=0)
    norm = np.linalg.norm(avg_vector)
    return (avg_vector / norm).tolist() if norm > 0 else avg_vector.tolist()

# --- Interactive UI --- 
query_input = widgets.Text(value='Do you have a benefits policy change notice from HR?', description='Query:', layout=widgets.Layout(width='95%'))
mode_selector = widgets.RadioButtons(options=['A. Full Colpali Search', 'B. KNN Search Only', 'C. KNN + Rescore'], description='Search Mode:')
generate_button = widgets.Button(description="Generate Agent Command")
output_area = widgets.Output()

def generate_agent_command(b):
    output_area.clear_output()
    query_text = query_input.value
    search_mode = mode_selector.value
    
    with output_area:
        print("Generating vectors and Elasticsearch query...")
        query_multi_vectors = create_colqwen_query_vectors(query_text, model, processor)
        es_query_body = {}
        index_name = ""

        if search_mode.startswith('A.'):
            index_name = "colqwen-rvlcdip-demo-part1"
            es_query_body = {"size": 5, "query": {"script_score": {"query": {"match_all": {}}, "script": {"source": "maxSimDotProduct(params.query_vector, 'colqwen_vectors')", "params": {"query_vector": query_multi_vectors}}}}}
        else:
            index_name = "colqwen-rvlcdip-demo-part2"
            query_avg_vector = to_avg_vector(query_multi_vectors)
            knn_query = {"field": "colqwen_avg_vector", "query_vector": query_avg_vector, "k": 200, "num_candidates": 500}
            if search_mode.startswith('B.'):
                es_query_body = {"size": 5, "knn": knn_query}
            else: # C. KNN + Rescore
                rescore_def = {"window_size": 50, "query": {"rescore_query": {"script_score": {"query": {"match_all": {}}, "script": {"source": "maxSimDotProduct(params.query_vector, 'colqwen_vectors')", "params": {"query_vector": query_multi_vectors}}}}, "query_weight": 0.0, "rescore_query_weight": 1.0}}
                es_query_body = {"size": 5, "knn": knn_query, "rescore": rescore_def}
        
        # Pretty print the JSON query
        query_json_string = json.dumps(es_query_body, indent=2)
        
        # Generate the final prompt for the AI Agent
        agent_prompt = f"""@elasticsearch-mcp-server search index {index_name} with query: {query_json_string}"""
        
        print("Done!\n")
        display(Markdown("**Copy the following command and paste it into Claude Desktop:**"))
        display(Markdown(f"```\n{agent_prompt}\n```"))

generate_button.on_click(generate_agent_command)
display(widgets.VBox([query_input, mode_selector, generate_button, output_area]))